## The Farkle Simulator

In [1]:
import random
from collections import Counter

class Farkle(object):

    def __init__(self, dragon_health=3, starting_soldiers=0):
        self.dragon_health = dragon_health
        self.soldiers_lost = 0
        if starting_soldiers <= 0:
            self.soldier_limit = float("inf")
        else:
            self.soldier_limit = starting_soldiers
        self.dice_in_pool = 6
        self.damage_die = [
            0,
            0,
            0,
            0,
            1,
            2,
        ]
        self.damage_done = 0
        self.current_roll = []

        # Scoring
        ## Functions
        self.scoring_functions = [
            self.__straight,
            self.__six_of_a_kind,
            self.__three_pairs,
            self.__four_of_a_kind,
            self.__three_of_a_kind,
            self.__fives_and_ones,
        ]
        ## Values
        self.straight_score = 3000
        self.six_of_a_kind_score = 2500
        self.three_pairs_score = 2000
        self.four_of_a_kind_score = 1500

        self.three_of_a_kind_scores = {
            1: 1000,
            2: 200,
            3: 300,
            4: 400,
            5: 500,
            6: 600,
        }

        self.ones_score = 100
        self.fives_score = 50
        self.result = None

        # Run the experiment
        self.__run()

    def __run(self):
        # Run the experiment
        self.runs = 0
        while True:
            self.runs += 1
            old_damage = self.damage_done
            #print "Roll with", self.dice_in_pool, "dice"
            result = self.__roll()
            # Test in the following order:
            # 1. Win
            # 2. Out of soldiers
            # 3. Farkle
            #print "Soldiers left:", self.soldier_limit - self.soldiers_lost
            #print "Damage done:", self.damage_done
            #print
            if self.damage_done >= 3:
                self.result = "Win"
                break
            elif self.soldiers_lost >= self.soldier_limit:
                self.result = "Out of soldiers"
                break
            # If we score damage, we keep going if we Farkle
            elif result is None and old_damage == self.damage_done:
                self.result = "Farkle"
                break

    def __roll(self):
        self.current_roll = []
        # When the pool has run out, reset it
        if self.dice_in_pool == 0:
            self.dice_in_pool = 6

        for _ in range(self.dice_in_pool):
            self.current_roll.append(random.randrange(1, 7))

        self.current_roll.sort()
        #print "\tRoll:", self.current_roll
        self.damage_done += random.choice(self.damage_die)
        soldiers_lost = self.__score()

        # If we Frakle, game over
        if soldiers_lost is None:
            return None

        # Otherwise
        self.soldiers_lost += soldiers_lost
        return True

    def __score(self):
        soldiers_lost = 0
        for f in self.scoring_functions:
            result = f(self.current_roll)
            if result is not None:
                (lost, self.current_roll) = result
                soldiers_lost += lost
                #print "\t\t", self.current_roll

        # If no soldiers were lost, it is a frakle, game over!
        if not soldiers_lost:
            return None

        # After removing scoring dice, update the number of dice in the pool
        self.dice_in_pool = len(self.current_roll)

        return soldiers_lost

    def __straight(self, roll):
        if len(roll) == 6 and roll == range(1,7):
            #print "\t\tStraight"
            return (self.straight_score, [])

    def __six_of_a_kind(self, roll):
        if len(roll) == 6:
            value = roll[0]
            if sum(roll) == 6 * value:
                #print "\t\tSix"
                return (self.six_of_a_kind_score, [])

    def __three_pairs(self, roll):
        if len(roll) == 6:
            pair0 = roll[0] == roll[1]
            pair1 = roll[2] == roll[3]
            pair2 = roll[4] == roll[5]
            if pair0 and pair1 and pair2:
                #print "\t\tPairs"
                return (self.three_pairs_score, [])

    def __four_of_a_kind(self, roll):
        found = False
        # Find if there are any four of a kinds
        if len(roll) >= 4:
            c = Counter(roll)
            for value, count in c.iteritems():
                if count >= 4:
                    found = True
                    break

        if not found:
            return None

        #Remove the scoring dice
        for _ in range(4):
            roll.remove(value)

        #print "\t\tFour"
        return (self.four_of_a_kind_score, roll)

    def __three_of_a_kind(self, roll):
        found = False
        # Find if there are any four of a kinds
        if len(roll) >= 3:
            c = Counter(roll)
            for value, count in c.iteritems():
                if count == 3:
                    found = True
                    break

        if not found:
            return None

        #Remove the scoring dice
        for _ in range(3):
            roll.remove(value)

        #print "\t\tThree"
        return (self.three_of_a_kind_scores[value], roll)

    def __fives_and_ones(self, roll):
        soldiers_lost = 0
        # Sum up score
        for value in roll:
            if value == 1:
                soldiers_lost += self.ones_score
            if value == 5:
                soldiers_lost += self.fives_score

        if not soldiers_lost:
            return None

        # Remove values
        roll[:] = [v for v in roll if (v != 1 and v != 5)]

        #print "\t\tFives and Ones"
        return (soldiers_lost, roll)

In [2]:
RUNS = 1000000
START = 50
STOP = 10050

## Simulate the Expectation Value

This code figures out on average how many soldiers are lost when defeating the dragon.

*This will run in O(Minutes).*

In [3]:
final = {}

ev_results = []
runs = []
for _ in range(RUNS):
    f = Farkle()
    if f.result == "Win":
        ev_results.append(f.soldiers_lost)
        runs.append(f.runs)

In [4]:
import cPickle as pickle

with open('./expectation_'+str(RUNS)+'.pkl', 'w') as f:
    pickle.dump(ev_results, f)
    
with open('./runs_'+str(RUNS)+'.pkl', 'w') as f:
    pickle.dump(runs, f)

## Simulate the Outcomes

This code figures out how often each endgame state is reached as a function of the number of soldiers.

*This will run in **O(Hours)**.*

In [ ]:
final = {}
for soldiers in xrange(START, STOP, 50):
    results = []
    for _ in range(RUNS):
        f = Farkle(starting_soldiers=soldiers)
        results.append(f.result)
    
    c = Counter(results)
    final[soldiers] = c

In [16]:
import cPickle as pickle

with open('./outcomes_'+str(RUNS)+'.pkl', 'w') as f:
    pickle.dump(final, f)